In [2]:
import json
import pprint
import pandas as pd
import numpy as np
import os
import subprocess
import pexpect
import sys
import time
import tldextract
import re
import pickle

In [2]:
usuarioBlackbird = 
senhaBlackbird = 
filesBlackbird = [#'/scratch4/leandro/data/fn/pickles/top-10-perserver-20210817.csv',
                  '/scratch4/leandro/data/fn/pickles/top-10-perserver-20210825.csv',
                   '/scratch4/leandro/data/fn/compiled_trusty_news_portals.txt',
                   '/scratch4/leandro/data/fn/url2title.csv']
destinationLocal = './data/'

SyntaxError: invalid syntax (<ipython-input-2-03c44ec05a5b>, line 1)

In [4]:
#Baixa os arquivos listados em filesBlackbird APENAS SE já não estiverem no diretorio /data
for file in filesBlackbird:
    if not os.path.isfile('data/' + file.split('/')[-1]):
        child = pexpect.spawn("bash")
        child.timeout= 300
        child.sendline("scp -P2512 -r " +
                              usuarioBlackbird+
                              "@blackbird.dcc.ufmg.br:"+
                              file+ 
                              " "+
                             destinationLocal)
        child.logfile_read = sys.stdout
        child.expect("password:")
        child.sendline(senhaBlackbird)
        child.expect("100%")
        
        time.sleep(2)
        child.close() 
        time.sleep(2)

NameError: name 'filesBlackbird' is not defined

In [5]:
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_rows', 200)

In [6]:
#Carrega as URLs das 10 noticias mais compartilhadas dos sites de baixa credibilidade em um dataframe
dfUrlFalse = pd.read_csv('data/top-10-perserver-20210825.csv', delimiter='|')

In [7]:
print(dfUrlFalse.shape)
print(dfUrlFalse.describe())
print(dfUrlFalse.info())
dfUrlFalse.head(10)

(2045, 3)
            amount
count  2045.000000
mean     55.547677
std     112.857622
min       1.000000
25%       8.000000
50%      23.000000
75%      61.000000
max    2910.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2045 entries, 0 to 2044
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   server  2045 non-null   object
 1   url     2045 non-null   object
 2   amount  2045 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 48.1+ KB
None


,server,url,amount
0,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/noticias/31794/urgente-comandantes-do-alto-patamar-das-forcas-armadas-se-unem-e-lancam-forte-nota-contra-o-tse-equottem-algo-a-esconderequot,592
1,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/noticias/16821/a-pec-que-extingue-a-pec-da-bengala-ja-esta-na-ccj-e-hora-da-sociedade-agir,512
2,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/noticias/15866/governo-zera-tarifas-de-importacao-para-medicamentos-contra-cancer-e-hiv,498
3,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/noticias/31511/grave-randolfe-convoca-equotexercito-de-stedileequot-o-mst-para-derrubar-bolsonaro-veja-o-video,493
4,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/noticias/31713/avante-a-mentira-jamais-sobrepujou-a-verdade,476
5,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/noticias/13444/gasolina-cai-pelo-terceiro-mes-consecutivo-e-inflacao-de-fevereiro-e-a-menor-desde-o-plano-real,461
6,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/noticias/31382/urgente-advogado-de-daniel-silveira-rompe-o-silencio-lanca-dura-nota-e-dispara-equotmoraes-esta-interferindo-na-camara-e-na-pgrequot,441
7,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/noticias/10218/finalmente-alguem-escreveu-aquilo-que-todos-que-acreditam-em-lula-precisam-conhecer,432
8,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/noticias/14804/confianca-e-tudo-scania-hyundai-gm-carrefour-e-honda-anunciam-investimentos-no-brasil,422
9,jornaldacidadeonline.com.br,https://www.jornaldacidadeonline.com.br/noticias/14580/o-parlamento-e-a-causa-da-corrupcao-e-nao-a-consequencia,421


In [8]:
#Carrega e pre-processa os dominios de alta credibilidade COMO LISTADO na anj https://www.anj.org.br/associados/)
#O pre-processamento consiste em:
    #(1) Pra cada listagem, transforma em subdominio.dominio.sufixo
          #Ou seja, 
          #bbc.com/portuguese vira bbc.com
          #theintercept.com/brasil/ vira theintercept.com
          #uol.com.br/ vira uol.com.br
    #(2) Para listagens que tem subdominio e.g. g1.globo.com (g1 é subdominio) adiciona o dominio à lista de confiaveis
          #Ou seja, 
          #além de valor.globo.com teremos tambem globo.com na lista de fonte confiaveis
    #(3) Ordena a lista e retira duplicatas
          #A lista original da ANJ já vem com redundancias e.g. uol.com.br e uol.com.br/

with open('data/compiled_trusty_news_portals.txt', 'r') as file :
    filedata = file.readlines()

extraMainTrustyDomains = []
for i,dom in enumerate(filedata):
    ext = tldextract.extract(dom)
    
    if(len(ext.subdomain)>0):
        filedata[i] = ext.subdomain + "." + ext.domain +'.'+ ext.suffix + "\n"
        extraMainTrustyDomains.append(ext.domain +'.'+ ext.suffix + "\n")
    else:
        filedata[i] = ext.domain +'.'+ ext.suffix + "\n"
            
filedata = filedata + extraMainTrustyDomains
filedata = sorted(list(set(filedata)))
    

with open('data/compiled_trusty_news_portals.txt', 'w') as file:
    file.write("".join(filedata))

trustyNewsDomains_ = open("data/compiled_trusty_news_portals.txt", "r")
trustyNewsDomains = pd.DataFrame([x.strip() for x in trustyNewsDomains_.readlines()], columns=['url'])

#Retira sites indesejados
trustyUndesirables = ['nytimes', 'bbc','terra', 'intercept'] 
trustyNewsDomains = trustyNewsDomains.drop(trustyNewsDomains[trustyNewsDomains['url'].str.contains('|'.join(trustyUndesirables))].index)
trustyNewsDomains = trustyNewsDomains.reset_index(drop=True)


In [9]:
print(trustyNewsDomains.shape)
print(trustyNewsDomains.describe())
print(trustyNewsDomains.info())
trustyNewsDomains.head(10)

(100, 1)
               url
count          100
unique         100
top     uai.com.br
freq             1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     100 non-null    object
dtypes: object(1)
memory usage: 928.0+ bytes
None


,url
0,acidadeon.com
1,acritica.com
2,agazetaderondoniadigital.com
3,agorarn.com.br
4,an.com.br
5,atarde.com.br
6,atribuna.com.br
7,avozdacidade.com
8,bopaper.com.br
9,clicrbs.com.br


In [ ]:
#Carrega a coluna URL do csv com todas as URLs expandidas pelo leandro e 
    #pre-processa para pegar apenas as URLs de sites de alta credibilidade
    
#O pre-processamento consiste em
     #(1) Consertar erros no csv: substitui aspas duplas por simples 
     #(2) Pega as URLs cujo dominio ou subdominio estão na lista da ANJ AMPLIADA conforme descrito acima

with open('data/url2title.csv', 'r') as file :
    filedata = file.read()
filedata = filedata.replace('"', "'")
with open('data/url2title.csv', 'w') as file:
    file.write(filedata)

dfUrlTrue_ = pd.read_csv('data/url2title.csv', delimiter='§', engine='python', usecols=['url'])

dfUrlTrue =[]
for x in list(dfUrlTrue_['url']):
    try:
        ext = tldextract.extract(x)
    except Exception as e:
        #print("Deu excessao no x:", x , "\n")
        continue
    if(ext.domain +'.'+ ext.suffix in list(trustyNewsDomains['url'])):
        dfUrlTrue.append(x)
    elif(ext.subdomain + "." + ext.domain +'.'+ ext.suffix in list(trustyNewsDomains['url'])):
        dfUrlTrue.append(x)
        
dfUrlTrue = pd.DataFrame(dfUrlTrue, columns = ['url'])

#Retira urls com substrings indesejadas
urlTrueUndesirables = ['.jpg','index.php','/tag/', '/tudo-sobre/', '/especiais/','/editoria/'] 
dfUrlTrue = dfUrlTrue.drop(dfUrlTrue[dfUrlTrue['url'].str.contains('|'.join(map(re.escape,urlTrueUndesirables)))].index)
dfUrlTrue = dfUrlTrue.reset_index(drop=True)


indexesToDrop = []
for index, row in dfUrlTrue.iterrows():
    url = row['url']
    ext = tldextract.extract(url)
    
    #Retira urls cujo path não tem pelo menos um subdiretorio x tal que /x/ seja composto por letras E numeros. 
    # Se todos forem apenas letras ou apenas numeros a url é eliminada
    try:
        pathList = url.split(ext.suffix + '/')[1].split('/')
    except Exception as e:
        indexesToDrop.append(index)
        continue
        #print("Deu excessao na url:" , url)
        #print(sys.exc_info()[2])
        
    onlyAlphaOrOnlyDigitCount = 0
    for subdir in pathList:
        if(subdir.isdigit() or subdir.isalpha()):
            onlyAlphaOrOnlyDigitCount += 1
    if(url[-1]=='/'):
        if(onlyAlphaOrOnlyDigitCount==len(pathList)-1):
            indexesToDrop.append(index)
            continue
    else:
        if(onlyAlphaOrOnlyDigitCount==len(pathList)):
            indexesToDrop.append(index)
            continue
            
    #Retira urls que são apenas o proprio site     
    if (url.endswith((ext.suffix, ext.suffix + "/"))):
        if(index not in indexesToDrop):
            indexesToDrop.append(index)
        
dfUrlTrue = dfUrlTrue.drop(indexesToDrop)
dfUrlTrue = dfUrlTrue.reset_index(drop=True)

dfUrlTrue = dfUrlTrue.sample(n=10000, random_state=10)



In [ ]:
print(dfUrlTrue.shape)
print(dfUrlTrue.describe())
print(dfUrlTrue.info())
dfUrlTrue.head(5)

In [ ]:
dfUrlTrue.sample(n=10).head(40).sort_values('url')

In [ ]:
from source_features.source_features import SourceFeaturesExtractor

In [ ]:
sourceExtractor = SourceFeaturesExtractor("8c4b00a0873a12e53b7a2384ad8eeb9f")

In [ ]:
#Eu to deixando essa seção aqui pra testar rapidamente modificações no calculo das features..
#Na hora da coleta tu sempre descobre algo que ta fazendo errado ou pode melhorar

# url = "https://www.correiodopovo.com.br/"
    
# data_ipstack = sourceExtractor.get_ipstack_data(url)
# print(data_ipstack)
# print("\n")

# data_asn = sourceExtractor.get_asn_data(data_ipstack['subdomain_ip'])
# print(data_asn)
# print("\n")

# data_meta = sourceExtractor.get_ip_vs_asn_data(data_asn, data_ipstack)
# print(data_meta)
# print("\n")

# data_network = sourceExtractor.get_network_data(url)
# print(data_network)
# print("\n")

In [ ]:
sourceColumns = [
                   'subdomain_ip', 'subdomain_ip_cc','subdomain_ip_is_brazil','subdomain_ip_is_us',
                   'subdomain_ip_latitude','subdomain_ip_longitude',
                   'subdomain_as_n','subdomain_as_cc',
                   'subdomain_ipcc_equal_ascc',
                   'domain_route_hops','domain_dns_caa_txt_count'
                 ]

In [ ]:
dfSourceFeatures = pd.DataFrame(columns = ['desinformacao_label', 'url'] + sourceColumns)

In [ ]:
#Criando esse df a mais pq muitas urls compartilham as mesmas source features ja que sao do mesmo site
#é imperativo: as features em data_network demoram 1 minuto POR INSTANCIA para serem coletadas
    #300mil minutos recalculando vs 400 minutos reutilizando
if os.path.isfile('dfSubdomainSourceFeatures22Apr2021.pkl'):
    dfSubdomainSourceFeatures = pd.read_pickle('dfSubdomainSourceFeatures22Apr2021.pkl')
else:  
    dfSubdomainSourceFeatures = pd.DataFrame(columns = ['desinformacao_label','subdomain'] + sourceColumns)

In [ ]:
import logging

In [ ]:
def generate_source_features(dfUrlX, desinformacao_label, logFile):
    newSubdomainCount = 0
    begin = time.time()
    
    LOG_FILENAME =  'logSourceFeatures' + logFile + '.log'
    logging.basicConfig(filename=LOG_FILENAME, filemode ='a',level=logging.WARNING)
    
    for index, row in dfUrlX.iterrows():

        #Esse if ta aqui só pra não calcular pra todos os dados o tempo todo enquanto desenvolvia o nb
        #Só comentar esse if pra percorrer todas as URLs calculando features
        #if(index < 50):
            url = row['url']
#             with open(LOG_FILENAME) as myfile:
#                 if url in myfile.read():
#                     continue
            dissectedUrl = tldextract.extract(url)
            subdomain = '.'.join(part for part in dissectedUrl if part)

            #print("Url:", url)
            #print("subdomain:", subd)

            dfSourceFeatures.at[len(dfSourceFeatures),'desinformacao_label'] = desinformacao_label
            dfSourceFeatures.at[len(dfSourceFeatures)-1,'url'] = url

            if(subdomain in list(dfSubdomainSourceFeatures['subdomain'])):
                features = dfSubdomainSourceFeatures[dfSubdomainSourceFeatures['subdomain'] == subdomain][0:1]
                for col in sourceColumns:
                    dfSourceFeatures.at[len(dfSourceFeatures)-1,col] = features[0:1][col].values[0]
            else:
                
                newSubdomainCount += 1
                print("Coletando dominio inedito:", subdomain, "#",newSubdomainCount)
                print("Elapsed time: ", time.time() - begin)
                try:
                    #IPStack data vai ser calculado em cima do subdominio
                    row_data_ipstack = sourceExtractor.get_ipstack_data(url)
                    #ASN data vai ser calculado em cima do IP encontrado do SUBDOMINIO (IP de g1.globo.com e não globo.com)
                    row_data_asn = sourceExtractor.get_asn_data(row_data_ipstack['subdomain_ip'])
                    row_data_meta = sourceExtractor.get_ip_vs_asn_data(row_data_asn, row_data_ipstack)
                    #Network data vai ser calculado em cima do DOMINIO (globo.com e não g1.globo.com)
                    row_data_network = sourceExtractor.get_network_data(url)


                    mergedDict = {**row_data_ipstack, **row_data_asn, **row_data_network,**row_data_meta}
                    dfSubdomainSourceFeatures.at[len(dfSubdomainSourceFeatures),'desinformacao_label'] = desinformacao_label
                    dfSubdomainSourceFeatures.at[len(dfSubdomainSourceFeatures)-1,'subdomain'] = subdomain

                    for col in sourceColumns:
                        dfSourceFeatures.at[len(dfSourceFeatures)-1,col] = mergedDict[col]
                        dfSubdomainSourceFeatures.at[len(dfSubdomainSourceFeatures)-1,col] = mergedDict[col]
                except Exception as e:
                    logging.exception(url)
                
                

In [ ]:
generate_source_features(dfUrlTrue,0,'urlTrue')

In [ ]:
print(dfSourceFeatures.shape)
print(dfSourceFeatures.describe())
print(dfSourceFeatures.info())
dfSourceFeatures.head(5)

In [ ]:
print(dfSubdomainSourceFeatures.shape)
print(dfSubdomainSourceFeatures.describe())
print(dfSubdomainSourceFeatures.info())
dfSubdomainSourceFeatures.head(5)

In [ ]:
generate_source_features(dfUrlFalse,1,'urlFalse')

In [ ]:
print(dfSourceFeatures.shape)
print(dfSourceFeatures.describe())
print(dfSourceFeatures.info())
dfSourceFeatures.head(5)

In [ ]:
print(dfSubdomainSourceFeatures.shape)
print(dfSubdomainSourceFeatures.describe())
print(dfSubdomainSourceFeatures.info())
dfSubdomainSourceFeatures.head(5)

In [ ]:
#shuffle no df
dfSourceFeatures = dfSourceFeatures.sample(frac=1).reset_index(drop=True)

dfSourceFeatures.to_pickle("./dfSourceFeatures28Aug2021.pkl")

In [ ]:
dfSubdomainSourceFeatures.to_pickle("./dfSubdomainSourceFeatures28Aug2021.pkl")

In [ ]:
# dfSourceFeatures['desinformacao_label']= dfSourceFeatures['desinformacao_label'].astype('bool')
# #dfSourceFeatures['url']= dfSourceFeatures['url'].astype('')

# #dfSourceFeatures['subdomain_ip']= dfSourceFeatures['subdomain_ip'].astype('')
# dfSourceFeatures['subdomain_ip_cc']= dfSourceFeatures['subdomain_ip_cc'].astype('category')
# dfSourceFeatures['subdomain_ip_is_brazil']= dfSourceFeatures['subdomain_ip_is_brazil'].astype('bool')
# dfSourceFeatures['subdomain_ip_is_us']= dfSourceFeatures['subdomain_ip_is_us'].astype('bool')
# dfSourceFeatures['subdomain_ip_latitude']= dfSourceFeatures['subdomain_ip_latitude'].astype('float')
# dfSourceFeatures['subdomain_ip_longitude']= dfSourceFeatures['subdomain_ip_longitude'].astype('float')
# #dfSourceFeatures['subdomain_as_n']= dfSourceFeatures['subdomain_as_n'].astype('')
# dfSourceFeatures['subdomain_as_cc']= dfSourceFeatures['subdomain_as_cc'].astype('category')
# dfSourceFeatures['subdomain_ipcc_equal_ascc']= dfSourceFeatures['subdomain_ipcc_equal_ascc'].astype('bool')
# dfSourceFeatures['domain_route_hops']= dfSourceFeatures['domain_route_hops'].astype('int')
# dfSourceFeatures['domain_dns_caa_txt_count']= dfSourceFeatures['domain_dns_caa_txt_count'].astype('int')

In [ ]:
# #dfSubdomainSourceFeatures['subdomain']= dfSubdomainSourceFeatures['subdomain'].astype('')

# #dfSubdomainSourceFeatures['subdomain_ip']= dfSubdomainSourceFeatures['subdomain_ip'].astype('')
# dfSubdomainSourceFeatures['subdomain_ip_cc']= dfSubdomainSourceFeatures['subdomain_ip_cc'].astype('category')
# dfSubdomainSourceFeatures['subdomain_ip_is_brazil']= dfSubdomainSourceFeatures['subdomain_ip_is_brazil'].astype('bool')
# dfSubdomainSourceFeatures['subdomain_ip_is_us']= dfSubdomainSourceFeatures['subdomain_ip_is_us'].astype('bool')
# dfSubdomainSourceFeatures['subdomain_ip_latitude']= dfSubdomainSourceFeatures['subdomain_ip_latitude'].astype('float')
# dfSubdomainSourceFeatures['subdomain_ip_longitude']= dfSubdomainSourceFeatures['subdomain_ip_longitude'].astype('float')
# #dfSubdomainSourceFeatures['subdomain_as_n']= dfSubdomainSourceFeatures['subdomain_as_n'].astype('')
# dfSubdomainSourceFeatures['subdomain_as_cc']= dfSubdomainSourceFeatures['subdomain_as_cc'].astype('category')
# dfSubdomainSourceFeatures['subdomain_ipcc_equal_ascc']= dfSubdomainSourceFeatures['subdomain_ipcc_equal_ascc'].astype('bool')
# dfSubdomainSourceFeatures['domain_route_hops']= dfSubdomainSourceFeatures['domain_route_hops'].astype('int')
# dfSubdomainSourceFeatures['domain_dns_caa_txt_count']= dfSubdomainSourceFeatures['domain_dns_caa_txt_count'].astype('int')



sandbox de EDA, ignorem

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
missing_values_table(dfSourceFeatures)

In [ ]:
dfSubdomainSourceFeatures.describe()

In [ ]:
dfSubdomainSourceFeatures.count()

In [ ]:
dfSourceFeatures.info()

In [ ]:
dfSubdomainSourceFeatures[dfSubdomainSourceFeatures['desinformacao_label'] == 1]['domain_dns_caa_txt_count'].mean()

In [ ]:
aCategorical = [
                   'subdomain_ip', 'subdomain_ip_cc',
                   'subdomain_as_n','subdomain_as_cc',
                 ]

In [ ]:
aNumerical = [
'subdomain_ip_is_brazil','subdomain_ip_is_us',
                   'subdomain_ip_latitude','subdomain_ip_longitude',
           
                   'subdomain_ipcc_equal_ascc',
                   'domain_route_hops','domain_dns_caa_txt_count'
                 ]
for feature in aNumerical:
    print("Feature:", feature)
    print("Media true:")
    print(dfSubdomainSourceFeatures[dfSubdomainSourceFeatures['desinformacao_label'] == 0][feature].mean())
    print("Media false:")
    print(dfSubdomainSourceFeatures[dfSubdomainSourceFeatures['desinformacao_label'] == 1][feature].mean())
    print("\n")

In [ ]:
# allSubs = []
# for index, row in dfUrlTrue.iterrows():
#     url = row['url']
#     dissectedUrl = tldextract.extract(url)
#     subdomain = '.'.join(part for part in dissectedUrl if len(part)>0)
#     allSubs.append(subdomain)
#     #print(len(allSubs))
# print(len(list(set(allSubs))))

# allSubs = []
# for index, row in dfUrlFalse.iterrows():
#     url = row['url']
#     dissectedUrl = tldextract.extract(url)
#     subdomain = '.'.join(part for part in dissectedUrl if len(part)>0)
#     allSubs.append(subdomain)
#     #print(len(allSubs))
# print(len(list(set(allSubs))))

# #Tem 108 sites true
# #292 sites fake

In [ ]:
# def lendo_dataset(localizacao_arquivo):
#     # Leitura de dados:
#     dados = []
#     with open(localizacao_arquivo, mode='r') as f:
#         for line in f:
#             dados.append(json.loads(line.strip()))

#     # Imprimindo primeiro 3 coletas de cada
#     for i in range(3):
#         pp = pprint.PrettyPrinter(indent=4)
#         pp.pprint(dados[i])
#         print('\n')


# # Lendo TrueNews
# print('\n---------------TRUE NEWS:\n')
# lendo_dataset('data/DATASET_MPMG-TrueNews_selected.txt')

# print('\n\n\n---------------FAKE NEWS:\n')
# # Lendo FakeNews
# lendo_dataset('data/DATASET_MPMG-FakeNews_matched.txt')
# dfTrue = pd.read_json('data/DATASET_MPMG-TrueNews_selected.json')
# dfFalse = pd.read_json('data/DATASET_MPMG-FakeNews_matched.json')
# print(dfTrue.columns)

# dfTrue.replace('', np.nan, inplace=True)
# dfFalse.replace('', np.nan, inplace=True)
# dfTrue.replace('NULL', np.nan, inplace=True)
# dfFalse.replace('NULL', np.nan, inplace=True)
#print(dfTrue.shape, dfFalse.shape)